In [1]:
import pandas as pd
from tqdm import tqdm
import pandas as pd
import re
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
og_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 일반정보/가공전/"
fn_path = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 일반정보/가공후/"

og_filepath = os.listdir(og_path)
fn_filepath = os.listdir(fn_path)

In [191]:
og_filepath = ['멕시코(yellow).csv', '스위스(yellow).csv']

In [205]:
import pandas as pd
from tqdm import tqdm

import pandas as pd
from tqdm import tqdm


def process_row(row, df, ru = False, es = False):
    column_dict = {"COMPANY NAME": row}
    keys = [("ADDRESS", ""), ("ADDRESS", "/n"), ("ADDRESS", "/r"), ("PHONE NUMBER", "/n"), 
            ("MOBILE PHONE", "/n"), ("WEBSITE", "/n"), ("DESCRIPTION", "/n"), 
            ("WORKING HOURS", "/n"), ("PRODUCTS & SERVICES", "/n"), 
            ("LISTED IN CATEGORIES", "/n"), ("KEYWORDS", "/n"), ("CONTACT PERSON", "/n")]        

    for key, replace_char in keys:
        if any([key in str(df[row][idx]) for idx in range(len(df[row]))]):
            column_dict[key] = [str(df[row][idx]).replace(key, "").replace(replace_char, "") 
                                for idx in range(len(df[row])) 
                                if key in str(df[row][idx])][0]
    
    specific_keys = [("ESTABLISHMENT YEAR", -1), ("EMPLOYEES", -1), 
                     ("COMPANY MANAGER", slice(2,None)), ("EMAIL", -1), 
                     ("FAX", -1), ("REGISTRATION CODE", -1), ("VAT REGISTRATION", -1)]
    
    for key, slice_val in specific_keys:
        if any([key in str(df[row][idx]) for idx in range(len(df[row]))]):
            column_dict[key] = [str(df[row][idx]).split()[slice_val] 
                                for idx in range(len(df[row])) 
                                if key in str(df[row][idx])][0]
    
    if any(["Related Companies to" in str(df[row][idx]) for idx in range(len(df[row]))]):
        column_dict["Related Companies"] = [str(df[row][idx]).split()[3:] 
                                            for idx in range(len(df[row])) 
                                            if "Related Companies to" in str(df[row][idx])][0]
    
    return column_dict

for jdc in og_filepath:
    if "yellow" in jdc:
        df = pd.read_csv(og_path + jdc)
        company_lst = df.columns.to_list()

        column_dicts = [process_row(row, df) for row in tqdm(company_lst)]

        new_df = pd.DataFrame(column_dicts)

        new_df.to_excel(f"{fn_path}{jdc.split('.')[0]}.xlsx", index=False)




  0%|          | 0/184 [00:00<?, ?it/s]

100%|██████████| 184/184 [00:00<00:00, 484.89it/s]


In [193]:
translation_ru_dict = {
    'АДРЕС': 'ADDRESS',
    'ТЕЛЕФОН': 'PHONE NUMBER',
    'МОБИЛЬНЫЙ ТЕЛЕФОН': 'MOBILE PHONE',
    'ВЕБ-САЙТ': 'WEBSITE',
    'КОНТАКТНОЕ ЛИЦО': 'CONTACT PERSON',
    'ГОД СОЗДАНИЯ': 'ESTABLISHED IN',
    'СОТРУДНИКИ': 'EMPLOYEES',
    'ЭЛЕКТРОННАЯ ПОЧТА': 'EMAIL',
    'OПИСАНИЕ': 'DESCRIPTION',
    'РАБОЧЕЕ ВРЕМЯ': 'WORK TIME',
    'PRODUCTS_SERVICES': 'PRODUCTS & SERVICES',
    'ЗАРЕГИСТРИРОВАННАЯ В КАТЕГОРИИ': 'LISTED IN CATEGORIES',
    'КЛЮЧЕВЫЕ СЛОВА': 'KEYWORDS',
    'Похожие компании': 'SIMILAR COMPANIES'
}
translation_es_dict = {
    "DIRECCIÓN": "ADDRESS",
    "NÚMERO DE TELÉFONO": "PHONE NUMBER",
    "TELÉFONO MÓVIL": "MOBILE PHONE",
    "SITIO WEB": "WEBSITE",
    "ESTABLECIMIENTO ANUAL": "ESTABLISH",
    "EMPLEADOS": "EMPLOYEE",
    "GERENTE DE EMPRESA": "CONTACT PERSON",
    "E-MAIL": "FAX",
    "DESCRIPCIÓN DE LA EMPRESA": "DESCRIPTION",
    "HORAS DE APERTURA": "WORK TIME",
    "ACTIVIDADES DE NEGOCIO": "LISTED IN CATEGORIES",
    "Empresas Relacionadas de SHOCK MULTIMEDIOS": "RELATED IN COMPANIES"
}


In [194]:
def lang_to_eng(df, lang):
    translation_dict = globals().get(f'translation_{lang}_dict')
    for column in df.columns:
        for row, value in enumerate(df[column]):
            for key, replace_value in translation_dict.items():
                try:
                    if key in value:
                        df[column][row] = value.replace(key, replace_value)
                except: pass
    return df

In [199]:
df = pd.read_csv(og_path + og_filepath[0])

In [201]:
lang_to_eng(df, "es")

,"VC Dynamic - Marketing, Outsourcing & Productions.",Bodegas Guardamex,Dr.Arturo Reyes Mares,Taxitel,ARTEPISOS DE MADERA,Boden Ecosuperficies,Courtyard By Marriott Leon at The Poliforum,OSVER RENTA,BOVÉ RESTAURANTE Y TIENDA ORGANICA,BRASIL 2000,...,EUROPLAST,EUROMODA,EUROMOTO,EVER GREEN SHOES,EVENTS PROVIDENCE,EVER GREEN,EXCLUSIVAS FINAS EN PIEL,EXCELENCIA,EXCITY,EXHIBICIONES DINAMICAS
0,"ADDRESS\nPresa tres mezquites, León, Guanajuat...","ADDRESS\nBlvd. Haciendas de León 2956, Col. Ru...","ADDRESS\nave.cerro gordo 512,col. villas del c...","ADDRESS\nPriv. Muérdago 101, El Palote, 37130,...","ADDRESS\nAlud 605-5, col. Peñitas, León, Guana...","ADDRESS\nBlvd. José María Morelos, San Ángel 5...","ADDRESS\nBoulevard Francisco Villa No 102, Leó...","ADDRESS\nCRISTIANOS NO. 128 S/N, VILLAS DE LA ...","ADDRESS\nManuel J. Clouthier 402, Torres Metro...","ADDRESS\nMariano Escobedo 1008, Leon Moderno, ...",...,"ADDRESS\nRUIZ CORTINEZ 57, LOMAS DE ATIZAPAN, ...","ADDRESS\nAV GUANAJUATO NO. 301, JARDINES DEL M...","ADDRESS\nCAMPESTRE 99, PEÑITAS, 37180, GUANAJU...","ADDRESS\nVERACRUZ 1804, CHAPALITA, 37340, GUAN...","ADDRESS\nVIA SAN JORGE NO. 210, BRISAS DEL CAM...","ADDRESS\nVERACRUZ NO. 1804, CHAPALITA, 37340, ...","ADDRESS\nBETANIA NO. 615 S/N, SAN FELIPE DE JE...","ADDRESS\nCIRCUITO PORTON NEGRO 146, PORTONES D...","ADDRESS\nMANUEL MUÑOZ LEDO 114-A, LOS PARAISOS...","ADDRESS\nSAN MARTIN NO. 116 S/N, OBREGON, 3732..."
1,MOBILE PHONE\n045 477 378 7360,PHONE NUMBER\n+52 477 613 7380,PHONE NUMBER\n477 713 28 91,PHONE NUMBER\n+52 477 770 2121\n+52 477 715 0097,PHONE NUMBER\n477 7177433,PHONE NUMBER\n+52 477 854 2808,PHONE NUMBER\n+52-477-295 9400,PHONE NUMBER\n477.777-1641,PHONE NUMBER\n(477)299-1069,PHONE NUMBER\n(477)712-7850 (477)715-4857,...,PHONE NUMBER\n00822-1243\n00825-1819,PHONE NUMBER\n477.779-7611,PHONE NUMBER\n477.718-6638,PHONE NUMBER\n477.762-6003\n477.762-7703,PHONE NUMBER\n477.780-8372\n477.718-7725,PHONE NUMBER\n477.762-7703,PHONE NUMBER\n477.772-0928,PHONE NUMBER\n477.780-8437,PHONE NUMBER\n477.773-9730,PHONE NUMBER\n477.636-7610\n477.363-7620
2,WEBSITE\nhttp://www.vcdynamic.com.mx,WEBSITE\nhttps://www.bodegasguardamex.com,MOBILE PHONE\n4774899855,WEBSITE\nhttps://www.taxitel.mx,WEBSITE\nhttp://www.artepisosdemadera.mex.tl,WEBSITE\nhttps://www.bodeneco.com,FAX\n+52-477-295 9499,MOBILE PHONE\n4774497613,WEBSITE\nhttp://www.bove.com.mx,WEBSITE\nhttp://www.brasil-2000.com,...,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA,MAPA DE LOCALIZACIÓN\n AÑADIR MAPA
3,ESTABLISH 2012,ESTABLISH 2023,WEBSITE\nhttp://www.drarturoreyes.com.mx,DESCRIPTION\nEn Grupo Taxitel ofrecemos soluci...,ESTABLISH 2012,ESTABLISH 2023,WEBSITE\nhttp://www.marriott.com/hotels/travel...,ESTABLISH 1994,FAX\nENVIAR SOLICITUD,FAX\nENVIAR SOLICITUD,...,DESCRIPTION\nEUROPLAST está localizado en León...,"DESCRIPTION\nEUROMODA está localizado en León,...","DESCRIPTION\nEUROMOTO está localizado en León,...",DESCRIPTION\nEVER GREEN SHOES está localizado ...,DESCRIPTION\nEVENTS PROVIDENCE está localizado...,DESCRIPTION\nEVER GREEN está localizado en Leó...,DESCRIPTION\nEXCLUSIVAS FINAS EN PIEL está loc...,DESCRIPTION\nEXCELENCIA está localizado en Leó...,"DESCRIPTION\nEXCITY está localizado en León, M...",DESCRIPTION\nEXHIBICIONES DINAMICAS está local...
4,EMPLOYEE 1-5,EMPLOYEE 1-5,ESTABLISH 2017,WORK TIME\n AÑADIR HORAS,EMPLOYEE 1-5,EMPLOYEE 1-5,ESTABLISH 2006,EMPLOYEE 1-5,DESCRIPTION\nBOVÉ RESTAURANTE Y TIENDA ORGANIC...,DESCRIPTION\nBRASIL 2000 está localizado en Le...,...,WORK TIME\n AÑADIR HORAS,WORK TIME\n AÑADIR HORAS,WORK TIME\n AÑADIR HORAS,WORK TIME\n AÑADIR HORAS,WORK TIME\n AÑADIR HORAS,WORK TIME\n AÑADIR HORAS,WORK TIME\n AÑADIR HORAS,WORK TIME\n AÑADIR HORAS,WORK TIME\n AÑADIR HORAS,WOR

In [204]:
df.to_csv(og_path + og_filepath[0], index=False)

In [21]:
country = {"australia" : "aus",
"canada" : "can",
"switzerlands" : "che",
"colombia" : "col",
"germany" : "deu",
"spain" : "esp",
"france" : "fra",
"united-kingdom" : "gbr",
"hong-kong" : "hkg",
"indonesia" : "idn",
"india" : "ind",
"italy" : "ita",
"japan" : "jpn",
"south-korea" : "kor",
"malaysia" : "mys",
"mexico" : "mex",
"netherlands" : "nld",
"singapore" : "sgp",
"thailand" : "tha",
"united-states" : "usa",
"vietnam" : "vnm",
"호주" : "aus",
"캐나다" : "can",
"스위스" : "che",
"콜롬비아" : "col",
"독일" : "deu",
"스페인" : "esp",
"프랑스" : "fra",
"영국" : "gbr",
"홍콩" : "hkg",
"인도네시아" : "idn",
"인도" : "ind",
"이탈리아" : "ita",
"일본" : "jpn",
"대한민국" : "kor",
"말레이시아" : "mys",
"멕시코" : "mex",
"네덜란드" : "nld",
"싱가포르" : "sgp",
"태국" : "tha",
"미국" : "usa",
"베트남" : "vnm"}

stock_name = {
"aus" : ["Australian Securities Exchange", "ASX", "English"],
"can" : ["Toronto Stock Exchange", "TSX", "English"],
"che" : ["SIX Swiss Exchange", "SIX", "German"],
"col" : ["Bolsa de Valores de Colombia", "BVC", "Spanish"],
"deu" : ["Frankfurt Stock Exchange", "FSE", "German"],
"esp" : ["Bolsa de Madrid", "BM", "Spanish"],
"fra" : ["Euronext Paris", "EP", "French"],
"gbr" : ["London Stock Exchange", "LSE", "English"],
"hkg" : ["Hong Kong Stock Exchange", "HKEX", "Cantonese"],
"idn" : ["Indonesia Stock Exchange", "IDX", "Indonesian"],
"ind" : ["National Stock Exchange", "NSE", "Hindi"],
"ita" : ["Borsa Italiana", "BI", "Italian"],
"jpn" : ["Tokyo Stock Exchange", "TSE", "Japanese"],
"kor" : ["Korea Exchange", "KRX", "Korean"],
"mys" : ["Kuala Lumpur Stock Exchange", "KLSE", "Malay"],
"mex" : ["Bolsa Mexicana de Valores", "BMV", "Spanish"],
"nld" : ["Euronext Amsterdam", "BA", "Dutch"],
"sgp" : ["Singapore Stock Exchange", "SGX", "English"],
"tha" : ["Stock Exchange of Thailand", "SET", "Thai"],
"usa" : ["New York Stock Exchange", "NYSE", "English"],
"vnm" : ["Ho Chi Minh City Stock Exchange", "HSX", "Vietnamese"]}

In [180]:
!pip install googletrans

In [184]:
from googletrans import Translator
import time

def translate_text(text):
    translator = Translator()
    translated = translator.translate(text, dest="en")
    time.sleep(1)
    return translated.text

In [8]:
TableDefaultColumns = pd.read_excel("TableDefaultColumns.xlsx", sheet_name=2)

In [9]:
fn_filepath = ['네덜란드(yellow).xlsx', ""]

In [11]:
faillist = []

for idx in fn_filepath:
    finished_df = pd.DataFrame(columns=list(TableDefaultColumns["표준_한글컬럼명"]))
    df = pd.read_excel(fn_path + idx)
    country_name = idx.split("(")[0]
    try: 
        country_code = country[country_name]
        language = stock_name[country_code][2]
    except:
        print(country_name)
        country_code = ""
        language = ""
    for row in tqdm(range(len(df))):
        append_dict = {}

        # 각각의 값들에 대해 예외 처리를 해줍니다.
        for key in ["COMPANY NAME", "ADDRESS", "PHONE NUMBER", "MOBILE PHONE", "WEBSITE", 
                    "DESCRIPTION", "PRODUCTS & SERVICES", "LISTED IN CATEGORIES", 
                    "CONTACT PERSON", "FAX"]:
            try:
                append_dict[key] = df[key][row]
            except: append_dict[key] = ""  # 문제가 발생하면 빈 문자열로 설정

        append_dict.update({
            "헤브론스타국가코드" : country_code,
            "현지언어국가명" : country_name,
            # "영문국가명" : country_name,
            "국가언어코드값" : country_code,
            "현지언어언어명" : language,
            "영문언어명" : language,
            "현지언어기업명" : append_dict["COMPANY NAME"],
            # "영문기업명" : translate_text(str(append_dict["COMPANY NAME"])),
            "현지언어한줄소개내용" : append_dict["DESCRIPTION"],
            "현지언어기업소개내용" : append_dict["DESCRIPTION"],
            # "영문한줄소개내용" : translate_text(str(append_dict["DESCRIPTION"])),
            # "영문기업소개내용" : translate_text(str(append_dict["DESCRIPTION"])),
            "기업대표전화번호" : append_dict["PHONE NUMBER"],
            "대표팩스번호" : append_dict["FAX"],
            "기업홈페이지URL" : append_dict["WEBSITE"],
            "현지언어담당자명" : append_dict["CONTACT PERSON"],
            # "영문담당자명" : translate_text(str(append_dict["CONTACT PERSON"])),
            "기업담당자전화번호" : append_dict["PHONE NUMBER"],
            "기업담당자이동전화번호" : append_dict["MOBILE PHONE"],
            "현지언어기업주소" : append_dict["ADDRESS"],
            "현지언어기업상세주소" : append_dict["ADDRESS"],
            # "영문기업주소" : translate_text(str(append_dict["ADDRESS"])),
            # "영문기업상세주소" : translate_text(str(append_dict["ADDRESS"])),
            "현지언어산업군명" : append_dict["LISTED IN CATEGORIES"],
            # "영문산업군명" : translate_text(str(append_dict["LISTED IN CATEGORIES"])),
            "현지언어주요제품내용" : append_dict["PRODUCTS & SERVICES"],
            # "영문주요제품내용" : translate_text(str(append_dict["PRODUCTS & SERVICES"])),
        })

        finished_df = finished_df.append(append_dict, ignore_index=True)

    finished_df.to_excel(f"C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 일반정보/적재전(가공전)/{country_name}.xlsx", index=False)

네덜란드


100%|██████████| 3000/3000 [02:50<00:00, 17.61it/s]


PermissionError: [Errno 13] Permission denied: 'C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 일반정보/가공후/'

In [90]:
max_rows_column = df.count().idxmax()
max_rows_column

'SHOCK MULTIMEDIOS'